In [33]:
from ACDC.random_walk_classifier import * 
from ACDC.cell_type_annotation import * 

In [34]:
import pandas as pd
import numpy as np
from collections import Counter
import pickle

channels = ['pSTAT1','pSRC','pFAK','pMEK','pMAPKAPK2','pSTAT5','pMKK4','pS6K','pp53','pNFKB',
            'pp38','pAMPK','pAKT','pERK','cyclinB1','pGSK3B','GAPDH','pMKK3MKK6','pPDK1',
            'pBTK','pp90RSK','RAB7','bcatenin','pSTAT3','pJNK','pMARCKS','pPLCG2','pHH3','pS6',
            'cleavedPARP','PCNA','ACTIN','pRB','p4EBP1']

path = 'data/CC/'

df = pd.read_csv(path + 'CC_all.csv.gz', sep=',', header = 0, compression = 'gzip')
#df = df[df.cell_type != 'NotGated']


table = pd.read_csv(path + 'CC_table.csv', sep=',', header=0, index_col=0)
table = table.fillna(0)

cts, channels = get_label(table)

#X0= np.arcsinh((df[channels].values - 1.0)/5.0)
X0= df[channels].values

In [35]:
idx2ct = [key for idx, key in enumerate(table.index)]
#idx2ct.append('unknown')

ct2idx = {key:idx for idx, key in enumerate(table.index)}
#ct2idx['unknown'] = len(table.index)
        
ct_score = np.abs(table.as_matrix()).sum(axis = 1)

## compute manual gated label
y0 = np.zeros(df.cell_type.shape)

for i, ct in enumerate(df.cell_type):
    if ct in ct2idx:
        y0[i] = ct2idx[ct]
    #else:
        #y0[i] = ct2idx['unknown']

In [36]:
from sklearn.metrics import accuracy_score, confusion_matrix
import phenograph
from sklearn.cross_validation import StratifiedKFold

n_neighbor = 10
thres = 0.5


In [37]:
import time
import scipy.io as sio 

skf = StratifiedKFold(y0, n_folds=5, shuffle=True, random_state=0)
result = []
score_final = []


process_time = []
c = 0
for tr, te in skf:
    print('%02d th batch' % c)
    if c == 1:
        break
    c += 1
    
    X = X0.copy()
    y_true = y0.copy()

    X = X[tr, :]
    y_true = y_true[tr]

    mk_model =  compute_marker_model(pd.DataFrame(X, columns = channels), table, 0.0)

    ## compute posterior probs
    tic = time.clock()
    score = get_score_mat(X, [], table, [], mk_model)
    score = np.concatenate([score, 1.0 - score.max(axis = 1)[:, np.newaxis]], axis = 1)    

    ## get indices     
    ct_index = get_unique_index(X, score, table, thres)
    
    ## baseline - classify events    
    y_pred_index = np.argmax(score, axis = 1)
    
    toc = time.clock()
    time0 = toc - tic
    
    
    
    ## running ACDC
    tic = time.clock()
    res_c = get_landmarks(X, score, ct_index, idx2ct, phenograph, thres)

    landmark_mat, landmark_label = output_feature_matrix(res_c, [idx2ct[i] for i in range(len(idx2ct))]) 

    landmark_label = np.array(landmark_label)

    lp, y_pred = rm_classify(X, landmark_mat, landmark_label, n_neighbor)

    process_time.append(toc-tic)
    
    res = phenograph.cluster(X, k=30, directed=False, prune=False, min_cluster_size=10, jaccard=True,
                        primary_metric='euclidean', n_jobs=-1, q_tol=1e-3)
    
    toc = time.clock()
    time1 = toc - tic
    
    
    ## running phenograph classification
    tic = time.clock()
    y_pred_oracle = np.zeros_like(y_true)
    for i in range(max(res[0])+1):
        ic, nc = Counter(y_true[res[0] == i]).most_common(1)[0]
        y_pred_oracle[res[0] == i] = ic
        
    score_final.append([accuracy_score(y_true, [ct2idx[c] for c in y_pred]), 
                    accuracy_score(y_true, y_pred_index), 
                    accuracy_score(y_true, y_pred_oracle)])
    
    toc = time.clock()
    time2 = toc - tic   
    
    
    result.append((y_true, y_pred, y_pred_index, y_pred_oracle))
    process_time.append((time0, time1, time2))
    
    #pickle.dump(result, open('processed_file/BMMC/event_classidication_BMMC.p', 'wb'))
    sio.savemat('processed_file/CC_ACDC/event_classidication_CC_all_5.mat',{'y_true':y_true,'y_pred_index':y_pred_index,'y_pred_oracle':y_pred_oracle,'X':X})

00 th batch
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 2.8573050498962402 seconds
Jaccard graph constructed in 2.5297043323516846 seconds
Wrote graph to binary file in 0.7644014358520508 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.759736
After 2 runs, maximum modularity is Q = 0.763162
After 22 runs, maximum modularity is Q = 0.765205
Louvain completed 42 runs in 28.019931316375732 seconds
PhenoGraph complete in 34.226642370224 seconds
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 0.4450254440307617 seconds
Jaccard graph constructed in 2.0961198806762695 seconds
Wrote graph to binary file in 0.2538032531738281 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.649045
After 3 runs, maximum modularity is Q = 0.652078
After 4 runs, maximum modularity is Q = 0.653475
Louvain completed 24 runs in 9.028716325

In [38]:
np.mean(score_final, axis = 0) # score of ACDC, score-based classification, phenograph classification

array([ 0.39371563,  0.41922388,  0.83215112])

In [39]:
score_final

[[0.39371562514362751, 0.41922387510915693, 0.83215111914574169]]

In [40]:
process_time

[-1.9957069525844418e-06,
 (0.23133750284023336, 661.0744019507747, 0.04922639271080698)]